In [17]:
# !pip install transformers datasets accelerate nvidia-ml-py3

In [18]:
import numpy as np
from datasets import Dataset


seq_len, dataset_size = 512, 512
dummy_data = {
    "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),
    "labels": np.random.randint(0, 1, (dataset_size)),
}
ds = Dataset.from_dict(dummy_data)
ds.set_format("pt")

In [19]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [20]:
print_gpu_utilization()

GPU memory occupied: 2684 MB.


In [21]:
import torch


torch.ones((1, 1)).to("cuda")
print_gpu_utilization()

GPU memory occupied: 2684 MB.


In [22]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained("tiiuae/falcon-7b-instruct").to("cuda")
print_gpu_utilization()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 from transformers import AutoModelForSequenceClassification                                  │
│   2                                                                                              │
│   3                                                                                              │
│ ❱ 4 model = AutoModelForSequenceClassification.from_pretrained("tiiuae/falcon-7b-instruct").     │
│   5 print_gpu_utilization()                                                                      │
│   6                                                                                              │
│                                                                                                  │
│ c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\transformers\models\auto\auto_factory.p │
│ y:456 in from_pretrained                                                                         │
│                                                                                                  │
│   453 │   │   │   if kwargs.get("torch_dtype", None) == "auto":                                  │
│   454 │   │   │   │   _ = kwargs.pop("torch_dtype")                                              │
│   455 │   │   │                                                                                  │
│ ❱ 456 │   │   │   config, kwargs = AutoConfig.from_pretrained(                                   │
│   457 │   │   │   │   pretrained_model_name_or_path,                                             │
│   458 │   │   │   │   return_unused_kwargs=True,                                                 │
│   459 │   │   │   │   trust_remote_code=trust_remote_code,                                       │
│                                                                                                  │
│ c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\transformers\models\auto\configuration_ │
│ auto.py:950 in from_pretrained                                                                   │
│                                                                                                  │
│   947 │   │   config_dict, unused_kwargs = PretrainedConfig.get_config_dict(pretrained_model_n   │
│   948 │   │   has_remote_code = "auto_map" in config_dict and "AutoConfig" in config_dict["aut   │
│   949 │   │   has_local_code = "model_type" in config_dict and config_dict["model_type"] in CO   │
│ ❱ 950 │   │   trust_remote_code = resolve_trust_remote_code(                                     │
│   951 │   │   │   trust_remote_code, pretrained_model_name_or_path, has_local_code, has_remote   │
│   952 │   │   )                                                                                  │
│   953                                                                                            │
│                                                                                                  │
│ c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\transformers\dynamic_module_utils.py:53 │
│ 5 in resolve_trust_remote_code                                                                   │
│                                                                                                  │
│   532 │   │   if has_local_code:                                                                 │
│   533 │   │   │   trust_remote_code = False                                                      │
│   534 │   │   elif has_remote_code and TIME_OUT_REMOTE_CODE > 0:                                 │
│ ❱ 535 │   │   │   signal.signal(signal.SIGALRM, _raise_timeout_error)                            │
│   536 │   │   │   signal.alarm(TIME_OUT_REMOTE_CODE)       

In [23]:
!nvidia-smi

Mon Jun 26 16:20:12 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA T1000 8GB             WDDM  | 00000000:01:00.0  On |                  N/A |
| 35%   42C    P8              N/A /  50W |   2464MiB /  8192MiB |     20%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [24]:
default_args = {
    "output_dir": "tmp",
    "evaluation_strategy": "steps",
    "num_train_epochs": 1,
    "log_level": "error",
    "report_to": "none",
}

In [25]:
from transformers import TrainingArguments, Trainer, logging

logging.set_verbosity_error()


training_args = TrainingArguments(per_device_train_batch_size=4, **default_args)
trainer = Trainer(model=model, args=training_args, train_dataset=ds)
result = trainer.train()
print_summary(result)

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: WARNING! libcuda.so not found! Do you have a CUDA driver installed? If you are on a cluster, make sure you are on a CUDA machine!
CUDA SETUP: Loading binary c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\bitsandbytes\libbitsandbytes_cpu.so...
argument of type 'WindowsPath' is not iterable
CUDA SETUP: Problem: The main issue seems to be that the main CUDA library was not detected.
CUDA SETUP: Solution 1): Your paths are probably not up-to-date. You can update them via: sudo ldconfig.
CUDA SETUP: Solution 2): If you do not have sudo rights, you can do the following:
CUDA SETUP: Solution 2a): Find the cuda library via: find / -name libcuda.so 2>/dev/null
CUDA SETUP: Solution 2b): Once the library is found add it to the LD_LIBRARY_PATH: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:FOUND_PATH_FROM_2a
CUDA SETUP: Solution 2c): For a permanent solution add the export from 2b in

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\transformers\utils\import_utils.py:1082 │
│ in _get_module                                                                                   │
│                                                                                                  │
│   1079 │                                                                                         │
│   1080 │   def _get_module(self, module_name: str):                                              │
│   1081 │   │   try:                                                                              │
│ ❱ 1082 │   │   │   return importlib.import_module("." + module_name, self.__name__)              │
│   1083 │   │   except Exception as e:                                                            │
│   1084 │   │   │   raise RuntimeError(                                                           │
│   1085 │   │   │   │   f"Failed to import {self.__name__}.{module_name} because of the followin  │
│                                                                                                  │
│ C:\Users\matan.s\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py:126 in        │
│ import_module                                                                                    │
│                                                                                                  │
│   123 │   │   │   if character != '.':                                                           │
│   124 │   │   │   │   break                                                                      │
│   125 │   │   │   level += 1                                                                     │
│ ❱ 126 │   return _bootstrap._gcd_import(name[level:], package, level)                            │
│   127                                                                                            │
│   128                                                                                            │
│   129 _RELOADING = {}                                                                            │
│ in _gcd_import:1050                                                                              │
│ in _find_and_load:1027                                                                           │
│ in _find_and_load_unlocked:1006                                                                  │
│ in _load_unlocked:688                                                                            │
│ in exec_module:883                                                                               │
│ in _call_with_frames_removed:241                                                                 │
│                                                                                                  │
│ c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\transformers\trainer.py:202 in <module> │
│                                                                                                  │
│    199                                                                                           │
│    200                                                                                           │
│    201 if is_peft_available():                                                                   │
│ ❱  202 │   from peft import PeftModel                                                            │
│    203                                                                                           │
│    204                                                                                           │
│    205 if is_accelerate_available():                                                             │
│                                                                                                  │
│ c:\Users\matan.s\Desktop\python\ai_env\lib\site-packages\pe